In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pickle
import numpy as np
import matplotlib.pyplot as plt

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

from qiskit import Aer, QuantumCircuit, transpile, pulse
from qiskit_aer.primitives import Sampler
from qiskit.utils import algorithm_globals
from qiskit.providers.fake_provider import *
import qiskit_aer.noise as noise
from qiskit.circuit.library import *

In [2]:
import Qubit_taper as Q

In [3]:
path_hamiltonian = 'hamiltonian/OHhamiltonian.txt'
path_noise_model = {
  'cairo': 'noise_model/fakecairo.pkl',
  'montreal': 'noise_model/fakemontreal.pkl',
  'kolkata': 'noise_model/fakekolkata.pkl'
}
path_seeds = 'algorithm_seeds/requiredseeds.txt'
dir_qasm = 'qasm'

## Extract the hamiltonian

In [4]:
try:
    with open(path_hamiltonian, 'r') as file:
        file_contents = file.read()
except FileNotFoundError:
    print(f"Can't find the file: {path_hamiltonian}")
except Exception as e:
    print(f"Can't read the file: {str(e)}")

In [5]:
hamiltonian = Q.extract(file_contents,12)
taper_ham, hatree = Q.taper_X_remove(hamiltonian,3,(5,4))
taper_ham_exchange = Q.Z_exchange(taper_ham)
g_ham = Q.group_ham(taper_ham_exchange)

In [6]:
print("Observation number before : ",len(taper_ham))
print("Observation number grouping : ",len(g_ham))

Observation number before :  101
Observation number grouping :  19


## Evaluation

### Custom sampler for exact shot counting

In [7]:
class CountingSampler(Sampler):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._count = 0

    def run(self, circuits, *args, **kwargs):
        if isinstance(circuits, list):
            self._count += len(circuits)
        else:
            self._count += 1
        return super().run(circuits, *args, **kwargs)

    @property
    def run_count(self):
        return self._count
    
    @property
    def shot_count(self):
        return self._count * self.options.shots

### Some helpful functions

In [8]:
def calculate_metric(my_value):
    ref_value = -74.38714627
    nuclear_repulsion_energy = 4.36537496654537
    result = my_value + nuclear_repulsion_energy
    error_rate = (1 - abs((ref_value - result) / ref_value)) * 100
    return error_rate

def calculate_pulse(circuit, backend):
    with pulse.build(backend) as my_program:
        pulse.call(circuit)
    return my_program.duration

def get_noise_model(file_path):
    with open(file_path, 'rb') as file:
        noise_model_dict = pickle.load(file)
    return noise.NoiseModel.from_dict(noise_model_dict)

In [9]:
from dataclasses import dataclass

@dataclass
class Device:
    noise_model: noise.NoiseModel
    backend: FakeBackend
    qubit_location: list

backend_map = {
  'cairo': Device(
      get_noise_model(path_noise_model['cairo']), 
      FakeCairo(), 
      [24, 0, 1, 23, 12, 10]
  ),
  'montreal': Device(
      get_noise_model(path_noise_model['montreal']), 
      FakeMontreal(), 
      [24, 0, 1, 23, 12, 10]
  ),
  'kolkata': Device(
      get_noise_model(path_noise_model['kolkata']), 
      FakeKolkata(), 
      [16, 0, 1, 14, 25, 24]
  ),
}

### Run the evaluation

In [10]:
with open(path_seeds, 'r') as file:
    seed_list = file.read().replace(',', '').split('\n')
seed_list = [int(seed) for seed in seed_list if seed]

shot = 47368 

In [11]:
result_list = []

for name, device in backend_map.items():
    print(f"Backend: {name}")

    path_qasm = os.path.join(dir_qasm, f'{name}_QASM')
    path_ref_qasm = os.path.join(dir_qasm, f'{name}_ref_QASM')

    circuit = QuantumCircuit.from_qasm_file(path_qasm)
    ref_circuit = QuantumCircuit.from_qasm_file(path_ref_qasm)
    
    ## calculate classcially (Cost O(N^2))
    ref_value_ideal = Q.hatree_value(hatree,taper_ham) 

    for seed in seed_list:
        print(f"  Seed: {seed}")

        algorithm_globals.random_seed = seed
        seed_transpiler = seed

        sampler_noisy = CountingSampler(
            backend_options = {
                'method': 'statevector',
                'device': 'CPU',
                'noise_model': device.noise_model,
            },
            run_options = {
                'shots': shot,
                'seed': seed,
            },
            transpile_options = {
                'seed_transpiler':seed_transpiler,
                'optimization_level':0,
            }
        )

        ## noisy expectation value at reference state (hartree)
        ref_value_noisy = Q.expectation_value_circ_err_mit(
            para=None,
            ansatz=ref_circuit,
            g_ham=g_ham,
            sampler=sampler_noisy,
            qubits=device.qubit_location, 
            noise_model=device.noise_model, 
            err_mit=True, 
            nearest_prob_dists=True
        )
        
        ## noisy expectation value at optimized point
        expval = Q.expectation_value_circ_err_mit(
            para=None,
            ansatz=circuit,
            g_ham=g_ham,
            sampler=sampler_noisy,
            qubits=device.qubit_location, 
            noise_model=device.noise_model, 
            err_mit=True, 
            nearest_prob_dists=True
        )
        
        # reference state error mitigation
        correct_factor = (ref_value_ideal - ref_value_noisy).item().real
        mitigated_value = expval + correct_factor

        print('shot: ', sampler_noisy.shot_count)
        print('    Exp value (noisy): ', expval)
        print('    Exp value (mitigated): ', mitigated_value)

        result = {
            'backend': name,
            'seed': seed,
            'expval': mitigated_value,
            'metric': calculate_metric(mitigated_value),
            'duration': calculate_pulse(circuit, device.backend),
            'shot': sampler_noisy.shot_count,
        }
        result_list.append(result)

Backend: cairo
  Seed: 20
shot:  1799984
    Exp value (noisy):  -78.56275612591322
    Exp value (mitigated):  -78.70870653629866
  Seed: 21
shot:  1799984
    Exp value (noisy):  -78.56275290866832
    Exp value (mitigated):  -78.70870681948914
  Seed: 30
shot:  1799984
    Exp value (noisy):  -78.56274649230312
    Exp value (mitigated):  -78.70877519518886
  Seed: 33
shot:  1799984
    Exp value (noisy):  -78.56267489245363
    Exp value (mitigated):  -78.70877793710113
  Seed: 36
shot:  1799984
    Exp value (noisy):  -78.56255327861443
    Exp value (mitigated):  -78.7086661259884
  Seed: 42
shot:  1799984
    Exp value (noisy):  -78.56250763177118
    Exp value (mitigated):  -78.7090072995144
  Seed: 43
shot:  1799984
    Exp value (noisy):  -78.56249710589834
    Exp value (mitigated):  -78.7090045574478
  Seed: 55
shot:  1799984
    Exp value (noisy):  -78.56176609563524
    Exp value (mitigated):  -78.70893523989469
  Seed: 67
shot:  1799984
    Exp value (noisy):  -78.562377

### Save (or load) the results

In [12]:
import json
with open('result.json', 'w') as file:
    json.dump(result_list, file, indent=4)

# with open('result.json', 'r') as file:
#     result_list = json.load(file)

### Calculate the average expectation value and final metric

In [13]:
expval = {
    'cairo': 0,
    'montreal': 0,
    'kolkata': 0,
}

for result in result_list:
    for backend in expval.keys():
        if result['backend'] == backend:
            expval[backend] += result['expval']

for backend in expval.keys():
    expval[backend] /= len(seed_list)

print(expval)

{'cairo': -78.70880023201202, 'montreal': -78.72368420157821, 'kolkata': -78.69275194853438}


In [14]:
for backend in expval.keys():
    metric = calculate_metric(expval[backend])
    print(f"Backend: {backend}")
    print(f"  Avg. expval: {expval[backend]}")
    print(f"  Metric: {metric} (%)")

Backend: cairo
  Avg. expval: -78.70880023201202
  Metric: 99.94122505469606 (%)
Backend: montreal
  Avg. expval: -78.72368420157821
  Metric: 99.96123384695724 (%)
Backend: kolkata
  Avg. expval: -78.69275194853438
  Metric: 99.91965105396831 (%)
